### Модель 1

Загружает тексты напрямую из базы
- Использует встроенный embedding-механизм BERTopic (через SentenceTransformers)
- Строит темы и подбирает топовые слова для каждой
- Сохраняет результат в CSV и модель

---
### Метрики и формула 
impot = w1 * sum(posts) + w2 * mean(reactions) + w3 * avg(views) + w4 * mean(sentiment) * mean(confidance)
- w1=0.4, w2=0.3, w3=0.2, w4=0.1
- posts — количество постов в теме (чем больше, тем важнее)
- reaction — вовлечённость аудитории
- views — просмотры
- sentiment — можно использовать для приоритета положительных/негативных тем
- mean_confidence — доверие к sentiment (опционально)


In [8]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import umap
from sklearn.preprocessing import MinMaxScaler

#  Проверка CSV 
CSV_PATH = "../data/bertopic_model_v1.csv"
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f" Файл {CSV_PATH} не найден")

df = pd.read_csv(CSV_PATH)
print(" CSV успешно загружен")
print(f"Всего тем: {df['topic'].nunique()}")

#  Нормализация метрик 
scaler = MinMaxScaler()
df[['n_posts_norm', 'mean_reaction_norm', 'avg_views_norm', 'mean_confidence_norm']] = scaler.fit_transform(
    df[['n_posts', 'mean_reaction', 'avg_views', 'mean_confidence']]
)

#  Формула важности 
w1, w2, w3, w4 = 0.4, 0.3, 0.2, 0.1
df['importance'] = (
    w1 * df['n_posts_norm'] +
    w2 * df['mean_reaction_norm'] +
    w3 * df['avg_views_norm'] +
    w4 * df['mean_confidence_norm'] * df['mean_sentiment'].abs()
)

top20_important = df.sort_values('importance', ascending=False).head(20)
top20_important['example_post_short'] = top20_important['example_post'].str[:200] + "..."

print("\n Топ-20 самых важных тем по комбинированной метрике (v1):")
print(top20_important[['topic', 'importance', 'n_posts', 'mean_reaction', 
                       'avg_views', 'mean_sentiment', 'example_post_short']].to_string(index=False))


 CSV успешно загружен
Всего тем: 579

 Топ-20 самых важных тем по комбинированной метрике (v1):
 topic  importance  n_posts  mean_reaction    avg_views  mean_sentiment                                                                                                                                                                                          example_post_short
    -1    0.637716    21227       0.220245 2.335831e+05       -0.314317                                                                                                                      Американский актер и режиссер Роберт Редфорд скончался в возрасте 89 лет, пишет NYT...
    60    0.468647      212       0.999528 6.443300e+05       -0.995283 В CS2 почти НЕ ОСТАЛОСЬ читеров — Гейб бахнул ядерный апдейт для античита VAC. Штука сносит ВСЕ софты для волхаков и аимов, а читерам активнее выдаёт баны. Форумы и Discord-серваки читеров захлестнули...
   456    0.451704       21       0.592143 1.663825e+06       -0.142857   Со

### Модель 2

Основная идея: учёт актуальности постов через recency weight
- Для каждого поста вычисляли days_ago — сколько дней прошло с публикации
- Применялось экспоненциальное затухание веса
Важнее свежие посты, старые почти не учитываются, поэтому weighted_views и другие метрики могут сильно отличаться.
---
### Метрики
- weighted_views Сумма просмотров с учётом “старости” постов
- mean_weighted_sentiment Средний сентимент с учётом затухания
- mean_weighted_reaction Средние реакции с учётом времени
- mean_confidence Средняя уверенность сентимент-модели
- n_posts Кол-во постов в теме (не взвешенное)

In [ ]:
import pandas as pd
import os

#  Проверка CSV 
CSV_PATH = "../data/bertopic_model_v2.csv"

if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f" Файл {CSV_PATH} не найден")

df = pd.read_csv(CSV_PATH)
print(" CSV успешно загружен")
print(f"Всего тем: {df['topic'].nunique()}")

#  Топ-20 тем по weighted_views 
top20 = df.sort_values("weighted_views", ascending=False).head(20).copy()

# Ограничиваем длину example_post для вывода
top20['example_post_short'] = top20['example_post'].str[:200] + "..."  # первые 120 символов

# Выводим 
print("\n Топ-20 самых важных тем по weighted_views:")
print(top20[['topic', 'n_posts', 'weighted_views', 
             'mean_weighted_sentiment', 'mean_weighted_reaction', 
             'example_post_short']].to_string(index=False))

 CSV успешно загружен
Всего тем: 565

 Топ-20 самых важных тем по weighted_views:
 topic  n_posts  weighted_views  mean_weighted_sentiment  mean_weighted_reaction                                                                                                                                                                                          example_post_short
    -1    20018    1.902550e+08                -0.009848                0.001980                                                           «Андор» — первый трейлер второго сезона. Продолжение лучшего сериала во вселенной «Звёздных войн» стартует 22 апреля. Второй сезон — последний...
    12      381    4.489853e+07                -0.262667                0.000000 Правящая Либерально-демократическая партия и оппозиционная Партия обновления Японии подписали соглашение о формировании коалиции. Таким образом, в Японии скоро может появиться первая в истории страны ...
    13      349    4.077046e+07                 0.263911       

### Модель 3
- Все посты учитываются одинаково, независимо от даты публикации
- Можно её потом загрузить и использовать для трансформации новых данных или визуализации
- Темы с большим количеством старых постов тоже попадают в топ
---
### Метрки
- topic Идентификатор темы (числовой, уникальный)
- mean_reaction Среднее количество реакций на посты в теме
- mean_sentiment Средний сентимент (от -1 до 1) по теме
- mean_confidence Средняя уверенность модели при оценке сентимента
- avg_views Среднее количество просмотров постов в теме
- n_posts Количество постов, относящихся к теме
---

In [ ]:
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px

CSV_PATH = "../data/bertopic_model_v3.csv"

if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"Файл {CSV_PATH} не найден")

df = pd.read_csv(CSV_PATH)
print("CSV успешно загружен")
print(f"Всего тем: {df['topic'].nunique()}")

#  Топ-20 по количеству постов 
top20 = df.sort_values("n_posts", ascending=False).head(20)
top20['example_post_short'] = top20['example_post'].str[:120] + "..."

print("\n Топ-20 тем по количеству постов (v3):")
print(top20[['topic', 'n_posts', 'avg_views', 'mean_reaction', 
             'mean_sentiment', 'example_post_short']].to_string(index=False))

#  Комбинированная метрика важности 
print("Считаем комбинированную метрику важности тем...")

scaler = MinMaxScaler()
df[['mean_reaction_norm', 'mean_sentiment_norm', 'avg_views_norm', 'n_posts_norm']] = scaler.fit_transform(
    df[['mean_reaction', 'mean_sentiment', 'avg_views', 'n_posts']]
)

# Веса признаков
w1, w2, w3, w4 = 0.3, 0.25, 0.2, 0.25

# Комбинированная метрика важности
df['importance'] = (
    w1 * df['avg_views_norm'] +
    w2 * df['mean_reaction_norm'] +
    w3 * (df['mean_confidence'] * df['mean_sentiment_norm'].abs()) +
    w4 * df['n_posts_norm']
)

# Сохраняем
out_path = "../data/bertopic_model_v3_importance.csv"
df.to_csv(out_path, index=False)
print(f"Сохранено с метрикой в {out_path}")

#  Топ-20 по важности 
top20_imp = df.sort_values("importance", ascending=False).head(20)
top20_imp['example_post_short'] = top20_imp['example_post'].str[:120] + "..."
print("\nТоп-20 самых важных тем:")
print(top20_imp[['topic', 'importance', 'avg_views', 'mean_reaction', 
                 'mean_sentiment', 'mean_confidence', 'n_posts', 
                 'example_post_short']].to_string(index=False))



✅ CSV успешно загружен
Всего тем: 566

🔥 Топ-20 тем по количеству постов (v3):
 topic  n_posts     avg_views  mean_reaction  mean_sentiment                                                                                                          example_post_short
    -1    21758 234038.053635       0.244387       -0.333303 Семье погибшего в результате ночных прилетов уроженца Липецка выплатят 500 тыс. рублей из средств благотворительного фон...
     0      858 419039.022145       0.000000       -1.000000                                              Россию совершенно не устраивает вступление Украины в НАТО, подчеркнул Путин...
     6      749 153708.353805      -0.496936       -1.000000 Делаем картинки за несколько секунд: Hailuo AI выпустили свой генератор изображений Image-01. Бесплатных кредитов хватит...
     4      725 262927.154483       0.090862       -0.964138 Poizon ОФИЦИАЛЬНО вышел на российский рынок — журналисты получили ответ от Shanghai Dewu Information Group Co Это компан

In [2]:
import pandas as pd
df = pd.read_csv('../data/bertopic_model_v3.csv')
df.head()

,topic,mean_reaction,mean_sentiment,mean_confidence,avg_views,n_posts,example_post
0,-1,0.244387,-0.333303,0.790868,234038.053635,21758,Семье погибшего в результате ночных прилетов у...
1,0,0.000000,-1.000000,0.810949,419039.022145,858,Россию совершенно не устраивает вступление Укр...
2,6,-0.496936,-1.000000,0.808006,153708.353805,749,Делаем картинки за несколько секунд: Hailuo AI...
3,4,0.090862,-0.964138,0.801292,262927.154483,725,Poizon ОФИЦИАЛЬНО вышел на российский рынок — ...
4,1,-0.517268,-1.000000,0.808103,126325.794118,646,В подмосковной Ивантеевке загорелся частный до...


In [5]:
df_1 = df[['topic', 'example_post']].copy()
df_1.to_csv('df_1.csv', index=False)

In [9]:
df = pd.read_csv('../data/bertopic_model_v3.csv') # оригинал куда надо добавить столбец subject
df_subjects = pd.read_csv("../data/df_1.csv") # там где есть столбец subject

# Объединяем по 'topic'
df_merged = df.merge(df_subjects[["topic", "subject"]], on="topic", how="left")

# Проверим результат
df_merged.to_csv('final_df.csv', index = False)

In [15]:
final_df = pd.read_csv('final_df.csv')
missing = final_df[final_df['subject'].isna() | (final_df['subject'] == "")]

# Смотрим сколько таких строк
print(f"Количество строк с пустым subject: {len(missing)}")

# Смотрим сами строки
missing.head(40)

Количество строк с пустым subject: 0


,topic,mean_reaction,mean_sentiment,mean_confidence,avg_views,n_posts,example_post,subject
